##Laila Wahedi
#Terrorist Alliance Model
##Goal: 
Alliances are both costly and beneficial to militant groups. Allied groups benefit from trade and returns to efficiency from joint production of violence. They share information and infrastructure. But maintaining an alliance requires resources, and sharing secure information entails vulnerability. We therefore expect to see groups being highly selective about the alliances they make. 
The topology of communities in the structure of the militant group alliance networks is highly varied. What explains the variation in terrorist network structure? This model will try to reproduce this variation by exploring the dynamics shaping alliance choice.

Given the set of theorized decision rules, does alliance formation produce efficient outcomes? In other words, are groups selecting into alliances that maximize their utility? Do they find communities that make them better off, or are the resulting alliances inefficient? Does this depend on time? 

##Justification:
Alliance formation is a network problem, because the actions made by any given group affect all of their partners. It is an agent based model because groups vary in their strength, and their interaction with any given group depends on their interaction with other agents. 
Alliance formation is not an equilibrium process. Groups continuously break and form alliances in the real world. An agent based model will help us to understand the dynamics by looking at snapshots, rather than looking for unreachable equilibria

##Outline:
-Implement counter terrorist

-Implement terrorist network, which contains terrorists

-Tune the model parameters so that costs and benefits are on the same scale

-Vary counterterrorist resources (risk), the strength of saturation costs, and the strength of network benefits

-Observe how various network attributes change over time. Do communities become more centralized, or are they flat? 

#Model Comonents
##Space:
The actors exist in a non-dimensional space in which they interact according to a random draw. They have the option to form alliances, after which they exist in a network, which goods and risks can flow across.

##Actors:
###Counter Terrorists:
Counter terrorists target transnational groups in two ways. First, they pick some constant number of new groups every period to investigate. Second, they use intelligence from previous operations to select groups. Allies of groups they targeted in the previous round have some probability of being targeted in the next round, with the probability dependent on the first group's strength, with stronger groups better safeguarding information. 

Counter terrorists have a list of targets, but also have limited resources. Among their targets, they select some capped number of groups to attack with their current intelligence. By the next period, the intelligence has expired, and they must select new potential targets.

###Terrorists:
Each period, terrorist groups with capacity to form additional alliances, (groups with remaining open alliance slots,) probabalistically interact with a set number of other groups, and consider an alliance with each. The group will select among the others that it interacted with in order to choose the group, if any, that provides the best utility increase. If the potential partner has no alliance openings, it will only form an alliance and discarding its least prefered group provides greater expected utility than the status quo. 

##Decisions:
Groups make alliance decisions based on their expected utility. Utility is calculated as the eigenvector centrality of the group's component scaled by its eigenvalue, and a tuning parameter. Eigenvector centrality was used because it represents the flow of broadcast network goods across a node. 

Groups also incur two types of cost. The first is a saturation cost: having an alliance is costly to maintain. This is equal to the a scaled degree centrality. The second is risk. I begin with an adjacency matrix weighted by the probability that a group will transfer intelligence to the counterterrorist when attacked. I take the eigenvector of the maximum eigenvalue corresponding to a group's component, and multiply it by the cost of being attacked if attacked 

##Initial conditions
###Counter-terrorists:
Counter terrorists begin with 0 targets. They randomly select a certain number to target, then select among them to attack
###Terrorists:
Terrorsits begin with no alliances, and no utility. Over time they beuild their first network, and this network shifts over time. 
#Model Parameters: (and their defaults)
-tune: These are tuning parameters that are used to bring all the costs and benefits in each terrorist groups utility function to be on the same scale. The tuning parameters are: 

    'strength': 1,

    'slots': 3,
    
    'transfer':.1,
    
    'degree':1,
    
    'eigen':3

-nGroups=150,: number of groups to include

-niters=1000: number of time periods in the model. Set iters because not just interested in equilibrium outcome. Want to look voer time. 

lambd=3,: the parameter for the exponential distribution of strength. 

cost=5: the cost imposed by a counter terrorist attack

CTresources=7: The number of attacks the counter terrorists can make

num_choices=5:The number of groups that a group choooses between when deciding whether to form an alliance. How many options are drawn for that group to choose among?

#Progress:
The model currently encompasses vulnerability and saturation costs, but does not encorporate commitment problems or competition, which may result in only a single type of alliance. My next step will be to introduce competition, and see how that changes alliance formation dynamics. 

In [1]:
import igraph as ig
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats
import numpy.linalg
import matplotlib.pyplot as plt 
import networkx as nt

#Counter terrorist class
Methods:

Target: Creates a list of new targets, using uniform random integers. Target selection is not related to group strength. Collects intelligence probabalistically from groups that were attacked previously.

Attack: randomly choose groups to attack from the target list.

transition: discard attack list, move mew targets into target list

run: a single counter terrorist turn in a time period in the model 

In [3]:
class CounterTerrorist(object):
    def __init__(self, resources, cost, num_groups,num_new):
        self.cost=cost
        self.terrorlist = num_groups
        self.new_targets = []
        self.targets=[]
        self.attack_list=[]
        self.resources=max(1,resources)
        self.num_new=num_new
    def target(self):
        #creates a list of new targets
        new_targets = np.random.randint(0,self.terrorlist-1,size=self.num_new).tolist()
        for group in self.attack_list:
            self.new_targets+=Terrorists.getNeighbors(group)
    def attack(self):
        try:#In case the target list is empty or smaller than resources-1.
            self.attack_list=np.random.shuffle(self.targets)[0:resources-1]
            Terrorists.impose_cost(self.attack_list, self.cost)
        except:
            self.attack_list=self.targets
            try:#in case it's empty
                Terrorists.impose_cost(self.attack_list,self.cost)
            except:
                pass
    def transition(self):
        self.targets=self.new_targets
        self.new_targets=[]
    def run(self):
        self.target()
        self.transition()
        self.attack()

#Terrorist class:
Methods:

distribute strength: Distributed according to an exponential distribution so few groups have high strength, most have low strength

distribute slots: This is related to strength. Using a poisson distribution with lambda = to a tuned density from the strength exponential distribution. This determines the number of alliances a group can make.

p_transfer: create a list of relinquishing intel about any given alliance partner if a group is attacked by the counter terrorist. This is a cdf of the normal distribution;This is proportional to strenghth.

run: a single turn. Groups with open slots randomly encounter potential alliance partners and choose to ally with the one that improves their utility the most, as long as that group is willing. The group could opt not to align with anyone, especially if none of the selected possible groups are willing to form an alliance. Groups are unwilling if it decreases their expected utility. 

utility function: Calculates the expected utility from partnering with a given group. It has one required input, and two optional inputs. The first input is the group use utility is being calculated. The seocnd is a potential ally, the third is a potentially excluded group. Note: right now, utility = degree distribution of partner. This is temporary and will change as I develop the theory

Get neighbors: get a numpy array of all of a groups network ties
form tie: form a tie between the groups in the first two arguments. Remove a tie between the first and the optional third.

impose cost: Impose the costs created by the counter terrorists. This is a place holder  presently, since utility isn't stored yet

In [4]:
class Terrorists(object):
    def __init__(self,num_groups,lambd,cost,tune):
        #lambd: lambda for the exponential distribution of strength
        #num
        self.n=num_groups
        self.lambd=float(lambd)
        self.cost=cost
        self.tune=tune
        self.adj=np.zeros((self.n,self.n))
        self.strength=self.distribute_strength()
        self.vertex_attrs={
            'strength':self.strength,
            'slots':self.distribute_slots(),
            #'utility':np.asarray(self.n*[0]),
            #'risk':np.asarray(self.n*[0]),
            }
        self.vertex_attrs['empty']=self.vertex_attrs['slots'][:]
        self.vertex_attrs['transfer']=self.p_transfer()
    def distribute_strength(self):
        return (np.random.exponential(scale=(1/self.lambd),size=self.n)*(self.tune['strength']))
    def distribute_slots(self):
        return(np.random.poisson(self.strength*(self.tune['slots'])))
    def p_transfer(self):
        #take the standardized strengths, and use a cdf to set the problability of
        #passing information to the counterterrorists when targeted. 
        #1- because stronger groups are less likely to share information.
        p1dim=(1-scipy.stats.norm.cdf(scipy.stats.zscore(self.vertex_attrs['strength'])+self.tune['transfer']))
        return p1dim.reshape(self.n,1)
    def run(self, num_choices):
        #Go through groups with open slots
        groups=np.nonzero(self.vertex_attrs['empty'])
        #randomize order
        np.random.shuffle(groups)
        for group in groups:
            #randomly select groups to interact with. Refer to these 
            #as partners below.
            options=np.random.randint(0,self.n,num_choices)
            #initialize list of options and the associated utility
            grp_opts=[]
            grp_util=[]
            #status quo utility for the group
            sq_grp=self.utilityFunctionc(group)
            for i, ptnr in enumerate(options):
                #status quo for partner, with no alliance shifts
                sq_ptnr=self.utilityFunctionc(ptnr)
                #if the partner has no free alliance slots, iteratively
                #compare utility of joining with the group while iteratively
                #dropping each of its current allies.
                if self.vertex_attrs['empty'][ptnr]==0:
                    #find the lowest value ally
                    ptnr_opts=[]
                    ptnr_util=[]
                    for j, ptnr2 in enumerate(self.getNeighbors(ptnr)):
                        ptnr_util.append(self.utilityFunctionc(ptnr,group,ptnr2))
                        ptnr_opts.append(ptnr2)
                    #Find the lowest utility partner's partner
                    ptnr_min_ptnr=np.argmax(np.asarray(ptnr_util))
                    #If replacing the min partner is better than the status quo, the partner 
                    #will accept and is therefore an option. 
                    if ptnr_util[ptnr_min_ptnr]>sq_ptnr:
                        grp_opts.append((ptnr,ptnr_opts[ptnr_min_ptnr]))
                        grp_util.append(ptnr_util[ptnr_min_ptnr])
                else:
                    u=self.utilityFunctionc(ptnr,group)
                    #If there is an empty slot
                    if u>sq_ptnr:
                        grp_opts.append(ptnr)
                        grp_util.append(sq_ptnr)
            try: 
                ax_ind=np.argmax(np.asarray(grp_util))
                if grp_util[max_ind]>sq_grp:
                    if self.vertex_attrs['empty'][ptnr]==0:
                        self.formTie(grp,grp_opts[max_ind][0],grp_opts[max_ind][1])
                    else:
                        self.formTie(group,grp_opts[max_ind])
            except:
                pass
    
    def utilityFunctionc(self,group,partner=False,rmv=False):
        #+eigenvector centrality(flow) -degree centrality(saturation cost). Note, needs tuning
        u1=float(0)
        u1-=self.getNeighbors(group).size*self.tune['degree']
        try:
            #account for the removed node
            type(rmv)#so it errors out if no rmv
            u1-=1 
        except:
            pass
        try:
            type(partner)#so it errors out if no partner
            u1+=1
        except: 
            pass
        #eigenvector centrality.
        tempMat=self.adj[:,:]
        try:
            tempMat[group,partner]=1
            tempMat[partner,group]=1
        except:
            pass
        try:
            tempMat[group,rmv]=0
            tempMat[rmv,group]=0
        except:
            pass
        vals,vecs=np.linalg.eig(tempMat)
        ind = vals.argsort()[::-1]
        vals=vals[ind]
        vecs=vecs[ind]
        #Take the element from the eigenvector associated with the largest eigenval that is not 0
        #do this to handle multiple componants 
        #use a try accept because if there are no ties, there will be no nonzero
        #eigenvector values, so there will be no value or risk to add. 
        try:
            ind1=max(np.nonzero(numpy.absolute(vecs[group,:]))[1])
            u1+=np.absolute(vecs[group,ind1])*vals[ind1]*self.tune['eigen']
            #Risk. Also a cost
            #Take eigenvector of risk matrix.
            rmat=tempMat*(self.vertex_attrs['transfer'])
            vals,vecs=np.linalg.eig(rmat)
            ind = vals.argsort()[::-1]
            vals=vals[ind]
            vecs=vecs[ind]
            u1-=vecs[group,ind1]*vals[ind1]*self.cost
        except:
            pass
        return u1
    def formTie(self,grp1,grp2,grp3=False):
        self.adj[grp1,grp2]=1
        self.adj[grp2,grp1]=1
        if grp3:
            self.adj[grp2,grp3]=0
            self.adj[grp3,grp2]=0
        self.vertex_attrs['empty'][grp1]-=1
        self.vertex_attrs['empty'][grp2]-=1
        if grp3:
            self.vertex_attrs['empty'][grp3]+=1
    def impose_cost(self,atutilitytack_list,cost):
        self.vertex_attrs['utility'][attack_list]-=self.cost
    def getNeighbors(self,group):
        try: 
            return np.nonzero(self.adj[group,:])[1]
        except:
            return np.asarray(np.nonzero(self.adj[group,:]))
        

#Model class
This class houses the previous two, and iteratively calls each of their run commands. 

In [10]:
class Model(object):
    def __init__(self, tune, nGroups=150, p=.01,niters=1000,lambd=3,cost=5,CTresources=7,new_targets=3,num_choices=5):
        self.n=nGroups
        self.niters=niters
        self.p=p
        self.outMats=[]
        self.lambd=lambd
        self.cost=cost
        self.tune=tune
        self.CTresources=CTresources
        self.num_choices=num_choices
        self.new_targets=new_targets
        #Initialize actors
        self.terrorists=Terrorists(self.n,self.lambd,cost,tune)
        self.ct=CounterTerrorist(self.CTresources, self.cost, self.n,self.new_targets)
    def run(self,niters):
        for i in range(0,self.niters-1):
            self.ct.run()
            self.terrorists.run(self.num_choices)
        return self.terrorists.adj

#Run the model

In [11]:
tuning={
    'strength': 1,
    'slots': 3,
    'transfer':.1,
    'degree':1,
    'eigen':3
}
m1=Model(tuning)
m1.run(1)

array([[ 0.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  1.,  1., ...,  1.,  1.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

#Output plan:
Once the model is complete and the parameters are tuned to the right order of magnitude/range, I will run a specific number of iterations before outputting the adjacency matrix and calculating the skew of the degree distribution for each community within the network, and will compare those distributions as the tuning parameters are adjusted to increase and decrease the weight on the different costs and benefits within the utility function, and as the counter terrorist resources go up and down. 
Once I figure out a way to calculate utility, hopefully without having to do it recursively, I will also output utility to assess the conditions under which alliance formation is less efficient. 

The tuning parameters need to be tuned to the right range, because outside that range, I only expect corner solutions. For example, eigenvector centrality, which is currently the payoff groups receive benefit, is small, so groups will never form any alliances since the saturation costs and risks will outweigh the benefits. If the tuning parameter is too high, a fully saturated network, (saturated instead of complete because it is subject to constrained edges per node,) with alliances always being preferable. 
